# Le classifieur de Bayes ☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Comprendre&nbsp;:
    * le problème qu'on souhaite résoudre en *machine learning*,
* Être sensibilisé aux notions de&nbsp;:
    * erreur de Bayes
    * classifieur de Bayes.
    


 ----

## I. Introduction

Reformalisons notre problème de classification. Soit $\mathcal{X}$ l'espace des données d'entrée et $\mathcal{Y}$ celui des données de sortie. On aurait par exemple $\mathcal{X}$ l'ensemble de toutes les photos et $\mathcal{Y}$ celui des labels "chien" et "chat". *A priori*, dans le cadre d'une application de détection "chien/chat", on ne s'attend pas à trouver n'importe quelle image (e.g. on ne s'attend pas à voir des photos de pizzas). L'idée est d'interpréter cela de manière probabiliste en considérons le couple de variables aléatoires suivant : 

$$X, Y\in\mathcal{X}\times\mathcal{Y}.$$

On appellera $\mu$ la mesure de $X$ dans le sens où étant donné $A\subseteq\mathcal{X}$, on a $\mathbb{P}(X\in A)=\mu(A)$ et $\eta(x)=\mathbb{P}(Y=1|X=x)$ (dans le cas de la classification binaire) la "probabilité a posteriori". La fonction $\eta$ revête d'autres formes lorsqu'on travaille sur d'autres problèmes (e.g. classification, top-k). Il est important de constater que $\mu$ et $\eta$ décrivent totalement le couple $X, Y$. En effet, soit $A\subseteq \mathcal{X}\times \mathcal{Y}$, nous avons :

$$\mathbb{P}(X, Y\in A)=\int_{A\cap \mathcal{X}\times \{1\}}\eta(x)d\mu+\int_{A\cap \mathcal{X}\times \{0\}}(1-\eta(x))d\mu.$$

Notre objectif est de construire une application $h:\mathcal{X}\mapsto\mathcal{Y}$ telle que les "prédictions" de $h$ soient bonnes dans le sens du risque suivant :

$$L(h)=\mathbb{P}(h(X)\neq Y)=\mathbb{E}\big[\textbf{1}\{h(X)\neq Y\}\big].$$

On veut que la probabilité que $h$ fasse des erreurs soit la plus faible possible.

Ne connaissant ni $\mu$ ni $\eta$, nous ne pouvons pas calculer $L$ et donc trouver le meilleur $h$. Habituellement, en *machine learning*, nous collectons des données afin d'estimer $L$ et construire le classifieur approprié. Cependant, dans ce *notebook*, nous supposerons que nous sommes un oracle et que nous connaissons le processus générateur.

## II. Le classifieur de Bayes

Supposons que nous connaissions $\eta$ (nous sommes un oracle). Quel est le meilleur classifieur que nous puissions construire ?

Il s'agit du classifieur suivant :

$$g^\star(x)=\begin{cases}1&\text{ si }\eta(x)\geq 0.5\\ 0&\text{ sinon.}\end{cases}$$

C'est ce qu'on appelle le classifieur de Bayes. C'est lui qui fait le moins d'erreurs (notons qu'il peut exister plusieurs classifieurs qui font aussi peu d'erreurs). On peut quantifier le risque atteint par ce classifieur :

$$L^\star=\mathbb{E}\big[\text{min}(\eta(X), 1-\eta(X))\big].$$

Si les labels sont déterministes (i.e. $\eta\in\{0, 1\}$), alors $L^\star=0$.

**<span style='color:blue'> Proposition</span>** 
$\not\exists g:\mathcal{X}\mapsto\mathcal{Y}$ tel que $L(g)<L^\star$. Dit autrement, on ne peut pas faire mieux que le classifieur de Bayes.



 ----
**<span style='color:orange'> Preuve</span>** 
Soit $x\in\mathcal{X}$ et $g:\mathcal{X}\mapsto\mathcal{Y}$ un classifieur quelconque. On a :

$$\begin{aligned}
\mathbb{P}(g(X)\neq Y|X=x)&=1-\mathbb{P}(g(X)=Y|X=x)\\
&= 1-(\mathbb{P}(Y=1, g(X)=1|X=x)+\mathbb{P}(Y=0, g(X)=0|X=x))\\
&=1-(\textbf{1}\{g(x)=1\}\mathbb{P}(Y=1|X=x)+\textbf{1}\{g(x)=0\}\mathbb{P}(Y=0|X=x))\\
&=1-(\textbf{1}\{g(x)=1\}\eta(x)+\textbf{1}\{g(x)=0\}(1-\eta(x))).
\end{aligned}$$

Ainsi, nous avons :

$$\begin{aligned}
\mathbb{P}(g^\star(X)\neq Y|X=x)-\mathbb{P}(g(X)\neq Y|X=x)&=\eta(x)(\textbf{1}\{g(x)=1\}-\textbf{1}\{g^\star(x)=1\})\\&\ \ \ +(1-\eta(x))(\textbf{1}\{g^\star(x)=0\}-\textbf{1}\{g(x)=0\})\\
&=(2\eta(x)-1)(\textbf{1}\{g^\star(x)=1\}-\textbf{1}\{g(x)=1\})\\
&\geq 0
\end{aligned}$$

Il suffit maintenant d'intégrer sur tous les "$x$" et le résultat est là.



 ----
Soit $h$ une fonction de $\mathcal{X}$ dans $\mathcal{Y}$. La question qu'on peut se poser est celle de l'excès de $h$ par rapport à $g^\star$. Comme on l'a vu dans la preuve précédente, nous avons&nbsp;:

$$
\begin{aligned}
\mathbb{P}(h(X)\neq Y)-L^\star&=\mathbb{E}\big[|2\eta(X)-1|\textbf{1}\{g^\star(X)\neq h(X)\}\big]\\
\end{aligned}$$

## III. Un petit exercice

Construisons un jeu de données de classification binaire.

In [ ]:
import numpy as np
from scipy.special import expit as sigmoid

beta = np.random.uniform(-10, 10, size=(2, 1))

def class_probability(x):
    logit = np.dot(x, beta)
    proba = sigmoid(logit)
    
    return proba

def dataset(n):
    X = np.random.uniform(-1, 1, size=(n, 2))
    y = np.random.binomial(n=1, p=class_probability(X))
    return X, y

X, y = dataset(50)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

**<span style='color:blue'> Exercice</span>** 
**En vous appuyant sur le code précédent (en réutilisant éventuellement du code fourni), implémentez le classifieur de Bayes.**



 ----

In [ ]:
def bayes_classifier(x):
    ####### Complete this part ######## or die ####################
    ...
    ###############################################################


In [ ]:
plt.figure(figsize=(12, 8))
XX, YY = np.mgrid[-1:1:500j, -1:1:500j]
predictions = bayes_classifier(np.stack([XX, YY], axis=2).reshape(500*500, 2))

plt.pcolormesh(XX, YY, predictions.reshape(500, 500), cmap=plt.cm.Paired, shading='auto')
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.title('Le classifieur de Bayes')
plt.show()


## IV. Quelques exercices

**<span style='color:blue'> Exercice</span>** 
**Considérons cette fois-ci le problème de classification multi-classes. Ici, $\mathcal{Y}=\{1, \ldots, C\}$. Notons $\eta_k(x)=\mathbb{P}(Y=k|X=x)$.**

* **Trouvez le classifieur de Bayes.**
* **Trouvez l'expression de l'erreur de Bayes.**



 ----



**<span style='color:blue'> Exercice</span>** 
**Considérons cette fois-ci le problème de classification multi-classes *Top-K*. À chaque prédiction exactement $K$ classes sont retournées. Cette fois-ci, nos prédictions se font dans l'espace $\mathcal{P}(\mathcal{Y})$ l'ensemble des parties de $\mathcal{Y}$. Notons $\eta_k(x)=\mathbb{P}(Y=k|X=x)$.**

* **Trouvez le classifieur de Bayes.**
* **Trouvez l'expression de l'erreur de Bayes.**



 ----


**<span style='color:blue'> Exercice (partie I)</span>** 
**Imaginons une situation où la variable aléatoire $X$ représente le nombre d'heures passées à revoir le cours chez soi par semaine. Soit la loi conditionnelle suivante&nbsp;:**

$$\eta(x)=\frac{x}{c+x},\ c>0,$$

**où $c$ est une constante. Celle-ci indique la probabilité que l'étudiant réussisse son année. Dit autrement, plus un étudiant travaille chez lui, plus la probabilité qu'il réussisse est grande. Construisez le classifieur de Bayes et déduisez-en l'erreur de Bayes.**



 ----

**<span style='color:blue'> Exercice (partie II)</span>** 
**En complétant l'exercice précédent, supposez les cas suivants et calculez une valeur de $L^\star$&nbsp;:**

* $\mu(\{c\})=\mathbb{P}(X=c)=1$,
* $X$ suit une loi uniforme sur $[0, 4c]$ (i.e. $\mu([a;b])=\int_a^b\frac{1}{b-a}dx$).



 ----

En pratique, nous aurions collecté un jeu de données $S_n=\{(X_i,Y_i)\}_{i\leq n}$ et nous aurions dû choisir notre classifieur à partir de ce dernier.